In [2]:
import pandas as pd

# Importando o dataset e reprocessando as labels

In [3]:
df = pd.read_table('/content/SMSSpamCollection', names=['label', 'sms_message'])
df.head()

,label,sms_message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df['label'] = df.label.map({'ham' : 0, 'spam' : 1})
df.head()

,label,sms_message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


# Bag of Words

O Bag of Words é uma forma de representar problemas envolvendo diversas frase, em que vemos cada frase ignorando sua sintaxe e gramática, e apenas como um multiconjunto de palavras, ou seja, apenas vemos quais palavras e quantas de cada palavra estão na frase, como se a pegassemos e a jogassemos num saco de qualquer forma.

Utiliza-se do Bag of Words pois algorítmos normalmente lidam melhor com números que com palavras, e esta é uma forma de se tomar dados numéricos das frases.

Farei a implementação do Bag of Words do zero numa lista exemplo como é pedido no material.

Primeiramente passamos todos os strings da lista para letra minúscula.

In [5]:
documents = ['Hello, how are you!',
             'Win money, win from home.',
             'Call me now.',
             'Hello, Call hello you tomorrow?']

lower_case_documents = []
for i in documents:
  i = i.lower()
  lower_case_documents.append(i)
print(lower_case_documents)

['hello, how are you!', 'win money, win from home.', 'call me now.', 'hello, call hello you tomorrow?']


Aí removemos a pontuação...

In [6]:
sans_punctuation_documents = []
import string

for i in lower_case_documents:
  i = i.translate(str.maketrans('', '', string.punctuation))
  sans_punctuation_documents.append(i)
    
print(sans_punctuation_documents)

['hello how are you', 'win money win from home', 'call me now', 'hello call hello you tomorrow']


Assim, tokenizamos os strings obtidos por os separar em cada palavra.

In [7]:
preprocessed_documents = []
for i in sans_punctuation_documents:
  preprocessed_documents.append(i.split(' '))
print(preprocessed_documents)

[['hello', 'how', 'are', 'you'], ['win', 'money', 'win', 'from', 'home'], ['call', 'me', 'now'], ['hello', 'call', 'hello', 'you', 'tomorrow']]


E contamos as frequências.

In [8]:
frequency_list = []
import pprint
from collections import Counter

for i in preprocessed_documents:
  frequency_list.append(Counter(i))
    
pprint.pprint(frequency_list)

[Counter({'hello': 1, 'how': 1, 'are': 1, 'you': 1}),
 Counter({'win': 2, 'money': 1, 'from': 1, 'home': 1}),
 Counter({'call': 1, 'me': 1, 'now': 1}),
 Counter({'hello': 2, 'call': 1, 'you': 1, 'tomorrow': 1})]


Feita esta implementação pedida, implementamos o Bag of Words pelo scikit-learn.

Primeiro importamos o CountVectorizer para vermos as palavras encontradas em "documents".

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()

count_vector.fit(documents)
count_vector.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['are',
 'call',
 'from',
 'hello',
 'home',
 'how',
 'me',
 'money',
 'now',
 'tomorrow',
 'win',
 'you']

Agora as organizamos em uma matriz mostrando a contagem das palavras em cada frase, em que cada linha é uma frase e cada coluna uma palavra.

In [10]:
doc_array = count_vector.transform(documents).toarray()
doc_array

array([[1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 2, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1]])

In [11]:
frequency_matrix = pd.DataFrame(doc_array,columns = count_vector.get_feature_names())
frequency_matrix

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,are,call,from,hello,home,how,me,money,now,tomorrow,win,you
0,1,0,0,1,0,1,0,0,0,0,0,1
1,0,0,1,0,1,0,0,1,0,0,2,0
2,0,1,0,0,0,0,1,0,1,0,0,0
3,0,1,0,2,0,0,0,0,0,1,0,1


# Conjuntos de treinamento e teste

Separarei os dados em conjuntos de teste e de treinamento para aplicar no modelo.

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['sms_message'], 
                                                    df['label'], 
                                                    random_state=1)

print('Number of rows in the total set: {}'.format(df.shape[0]))
print('Number of rows in the training set: {}'.format(X_train.shape[0]))
print('Number of rows in the test set: {}'.format(X_test.shape[0]))

Number of rows in the total set: 5572
Number of rows in the training set: 4179
Number of rows in the test set: 1393


E aplicamos o bag of words nestes dados.

In [13]:
count_vector = CountVectorizer()
training_data = count_vector.fit_transform(X_train)
testing_data = count_vector.transform(X_test)

# Implementando o teorema de Bayes do zero e especificidade e sensitividade

Dado um problema de decidir se um dado possui um valor positivo ou um valor negativo, e um teste que decide se o valor de algum dado dado é positivo ou negativo, a sensibilidade do teste é a probabilidade de que o teste nos retorne que o dado possui valor positivo caso o dado realmente possua valor positivo, e a especifidade do teste é a probabilidade de que o teste nos retorne um valor negativo no caso em que o valor do dado realmente seja negativo.

Estes são valores normalmente diferentes, pois, por exemplo, um teste que dá valor positivo para todos os dados recebidos terá uma sensibilidade de 100%, mas errará todos os valores negativos, dando uma especifidade de 0%. Idealmente quereriamos uma especifidade e uma sensibilidade ambos de 100%, mas nem sempre isto é possível, e muitas vezes procurar aumentar a sensibilidade diminui a especifidade e vice-versa. Assim, é bom procurar um balanço de ambos, ou decidir qual é mais importante de se valorizar.

Um exemplo em que a sensibilidade é mais adequada seria o de alguma doença letal, em que uma pessoa ou possui a doença ou não a possui, e é feito um teste para se descobrir isto, neste caso para evitar mortes com um tratamento adequado de quem realmente possui a doença, é mais importante que as chances de que o teste dê positivo para quem possui a doença sejam maiores.

Agora, se a doença não for letal ou decapacitante, como um pequeno resfriado, pode ser interessante para os médicos darem prioridade à especifidade, pois caso muitos pacientes apresentarem resultados positivos para uma doença que eles não possuem, os médicos poderão gastar muitos recursos e tempo com o tratamento destes pacientes de forma desnecessária.

Agora vamos à implementação pedida no material, no exemplo da diabetes, completandos os tópicos em ToDo desta seção.

In [14]:
# P(D)
p_diabetes = 0.01

# P(~D)
p_no_diabetes = 0.99

# Sensitivity or P(Pos|D)
p_pos_diabetes = 0.9

# Specificity or P(Neg/~D)
p_neg_no_diabetes = 0.9

# P(Pos)
p_pos = p_diabetes * p_pos_diabetes + p_no_diabetes * (1-p_neg_no_diabetes)
print('The probability of getting a positive test result P(Pos) is:',format(p_pos))

The probability of getting a positive test result P(Pos) is: 0.10799999999999998


In [15]:
p_diabetes_pos = (p_diabetes * p_pos_diabetes)/p_pos
print('Probability of an individual having diabetes, given that that individual got a positive test result is:\
',format(p_diabetes_pos)) 

Probability of an individual having diabetes, given that that individual got a positive test result is: 0.08333333333333336


In [16]:
# P(Pos/~D)
p_pos_no_diabetes = 0.1

# P(~D|Pos)
p_no_diabetes_pos = (p_no_diabetes * p_pos_no_diabetes)/p_pos
print('Probability of an individual not having diabetes, given that that individual got a positive test result is: {}',format(p_no_diabetes_pos))

Probability of an individual not having diabetes, given that that individual got a positive test result is: {} 0.9166666666666669


# Implementando o Naive Bayes do zero

Bem, aqui seguirei os passos pedidos pelo material para fazer os cálculos de probabilidade envolvendo as falas dos candidatos dos partidos políticos.

In [17]:
# P(J)
p_j = 0.5

# P(F/J)
p_j_f = 0.1

# P(I/J)
p_j_i = 0.1

p_j_text = p_j*p_j_f*p_j_i
print(p_j_text)

0.005000000000000001


In [18]:
# P(G)
p_g = 0.5

# P(F/G)
p_g_f = 0.7

# P(I/G)
p_g_i = 0.2

p_g_text = p_g*p_g_f*p_g_i
print(p_g_text)

0.06999999999999999


In [19]:
p_f_i = p_j_text + p_g_text
print('Probability of words freedom and immigration being said are: ', format(p_f_i))

Probability of words freedom and immigration being said are:  0.075


In [20]:
p_j_fi = p_j_text/p_f_i
print('The probability of Jill Stein saying the words Freedom and Immigration: ', format(p_j_fi))

The probability of Jill Stein saying the words Freedom and Immigration:  0.06666666666666668


In [21]:
p_g_fi = p_g_text/p_f_i
print('The probability of Gary Johnson saying the words Freedom and Immigration: ', format(p_g_fi))

The probability of Gary Johnson saying the words Freedom and Immigration:  0.9333333333333332


# Implementação do Naive Bayes com o scikit-learn

Agora utilizamos o pacote scikit-learn para utilizar o Naive Bayes para treinar com nossa data de treinamento e fazer previsões sobre a data de teste.

In [22]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(training_data,y_train)

MultinomialNB()

In [23]:
predictions = naive_bayes.predict(testing_data)

# Avaliando o modelo

Por fim, vemos se o modelo previu bem os dados que deixamos para teste.

In [24]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy score: ', format(accuracy_score(y_test, predictions)))
print('Precision score: ', format(precision_score(y_test, predictions)))
print('Recall score: ', format(recall_score(y_test, predictions)))
print('F1 score: ', format(f1_score(y_test, predictions)))

Accuracy score:  0.9885139985642498
Precision score:  0.9720670391061452
Recall score:  0.9405405405405406
F1 score:  0.9560439560439562


# Comparando algoritmos

Treinaremos agora com o algoritmo Decision Tree o mesmo conjunto de treino para compararmos o modelo recebido com o modelo feito pelo Naive Bayes.


In [40]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth = 10)
tree.fit(training_data, y_train)

predictionstree = tree.predict(testing_data)

print('Accuracy score: ', format(accuracy_score(y_test, predictionstree)))
print('Precision score: ', format(precision_score(y_test, predictionstree)))
print('Recall score: ', format(recall_score(y_test, predictionstree)))
print('F1 score: ', format(f1_score(y_test, predictionstree)))



Accuracy score:  0.9597989949748744
Precision score:  0.8603351955307262
Recall score:  0.8324324324324325
F1 score:  0.8461538461538461


Vemos que o Decision Tree, mesmo se mostrando um pouco decente, é pior que o Naive Bayes em todos os scores, de modo que o melhor modelo é o obtido pelo Naive Bayes. Especialmente, o precision score do Decision Tree é um tanto menor, indicando que estamos detectando como SPAM mais emails legítimos que podem ser importantes para o usuário que o modelo do Naive Bayes, o que é bastante indesejável.

Listarei os prós e contras de ambos os algoritmos.

Naive Bayes:

Contras:
- Devemos assumir independência dos dados.

- Caso uma categoria não esteja no conjunto de treinamento, o Naive Bayes derá uma frequência nula para esta categoria, ou seja, não conseguirá prever bem datas com categorias novas.

- Fazemos várias suposições sobre a distribuição do modelo que não são necessariamente verdadeiras.


Prós:

- É bom quando os datasets são pequenos e as features são independentes.

- E fácil de implementar.

- Pode ser paramétrico ou não paramétrico.

- Eficiente para classificação de documentos e filtro de SPAM.

Decision Tree

Contras:

- É sensível à rotação dos dados.

- Se não fixarmos uma semente, o mesmo dataset pode ser treinado em árvores diferentes, dando uma aleatoriedade maior para o algoritmo.

- Tende a se sobreajustar ao conjunto de treinamento.

Prós:

- É fácil de se interpretar e se explicar para outros, tendo também uma forma bem interessante de se visualizar.

- Pode ser usado tanto para regressão quanto para classificação de dados.

- Não precisa centralizar ou escalar os atributos.

- Pode lidar com features não lineares.

Por mais que a Decision Tree também possa ser usada para classificação de dados, o Naive Bayes o sobressaiu provavelmente pois o conjunto de dados não é muito grande, situação que é uma especialidade maior do Naive Bayes que da Decision Tree.